In [2]:
library(itecv) # should be loaded first
library(magrittr)
library(foreach)
library(doFuture)
library(zeallot)
library(tidyverse)

plotsize = function (x,y) options(repr.plot.width=x, repr.plot.height=y)

registerDoFuture()
plan(cluster, workers = 10)

Loading required package: future
Loading required package: iterators
Loading required package: parallel

Attaching package: ‘zeallot’

The following objects are masked from ‘package:future’:

    %->%, %<-%

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::accumulate() masks foreach::accumulate()
✖ tidyr::extract()    masks magrittr::extract()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ purrr::set_names()  masks magrittr::set_names()
✖ purrr::when()       masks foreach::when()


In [3]:
?sample_n

In [6]:
model_specs = list(
    gbm = list(
        tune_grid = expand.grid(
            n.trees = seq(1,501,40), 
            interaction.depth=3, 
            shrinkage = 0.1, 
            n.minobsinnode=3),
        extra_args = list(
            verbose=F, 
            bag.fraction=0.5)),
    glmnet = list(
        tune_grid = expand.grid(
            alpha=c(0,0.5,1),
            lambda=exp(seq(-5,2,0.4))),
        extra_args = list()))

metrics = list(
    mse = bundle_mse,
    wmse = bundle_wmse,
    r_objective = bundle_r_objective,
    match_mse = bundle_match_mse,
    trans_mse = bundle_trans_mse,
    match_mdc = bundle_match_mdc,
    trans_mdc = bundle_trans_mdc,
    ip_value = bundle_ip_value,
    dml_value = bundle_dml_value,
    gain = bundle_gain,
#     c_benefit = bundle_c_benefit,
    qini = bundle_qini,
    random = bundle_random)

replicates = 1:10
DGPs = powers_DGPs() %>% list(1:16) %>% 
    pmap(function(dgp, id) {
        dgp$id = id
        dgp
    })
c(itrain, ival, itest) %<-% make_indices(1000, 1000, 1000)

In [7]:
run_sim = function(replicate, DGP, model_specs, metrics, itrain, ival, itest) {
    data = create_data(DGP, length(c(itrain, ival, itest)))
    estimates = estimate_val_test(data, itrain, model_specs)
    val_bundle = learn_validation_auxiliaries(data, ival, model_specs, randomized=DGP$randomized)
    val_metrics = estimate_val_metrics(estimates, val_bundle, metrics, ival)
    test_metrics = calc_test_metrics(data, estimates, itest)
    
    inner_join(val_metrics, test_metrics, by="model") %>%
        mutate(replicate=replicate, dgp=DGP$id)
}

In [18]:
start_time <- Sys.time()

foreach(replicate = replicates, 
        .export=c("model_specs", "metrics", "itrain", "ival", "itest", "run_sim"),
        .combine = bind_rows,
        .packages = (.packages())) %:%
    foreach(DGP = DGPs, 
            .export=c("model_specs", "metrics", "itrain", "ival", "itest", "run_sim"),
            .combine = bind_rows,
            .packages = (.packages()))  %dopar%  {
        run_sim(replicate, DGP, model_specs, metrics, itrain, ival, itest) 
    } %>%
write_csv("data/experimental_results_test.csv")

end_time <- Sys.time()
end_time - start_time